# Sql

### Setup

In [ ]:
import mysql.connector as mysql
import pandas as pd

# connect to the database using 'connect()' method
db = mysql.connect(
    host = "w-util-MySQL.ad.ufl.edu",
    user = "fsoa_student",
    passwd = "FSOAStudent!",
    database="fsoa_impink"
)
# buffered cursor
cursor = db.cursor(buffered=True)

## Main ingredients

The main ingredients of a SQL query:
    
- Select: what to select, can include calculations
- From: input table, can be more than one
- Where: filter on the input table, also used to specify how to join
- Having: filter on the output table 
- Group By: repeats the 'select' for each partition of unique values of the group by (most used with aggregates)
- Order by: sorting


### Examples

In [ ]:
qry ='''
select shortname.gvkey, shortname.fyear, shortname.sale 
from funda shortname
where fyear >= 2018  and fyear <= 2022;
'''

In [ ]:
df = pd.read_sql(qry, db)
print('#records', len(df))
df.head()

In [ ]:
# same as (with one table in the from there is no need for a short name)
qry ='''
select gvkey, fyear, sale 
from funda 
where fyear >= 2018  and fyear <= 2022;
'''

In [ ]:
df = pd.read_sql(qry, db)
print('#records', len(df))

## Aggregate functions

As part of the select (and group by) you can do aggregate functions such as min, max, count, avg (average).

It is most useful if you use this with group by, otherwise you just get sample aggregates. For example, you probably want to have the mean ROA for each industry-fyear as opposed to a single mean ROA (one number) for the full sample.

In [ ]:
qry ='''
select avg (ni/at) as at_avg, avg (ni/sale) as roa_median
from funda 
where fyear >= 2018  and fyear <= 2022;
'''

In [ ]:
df = pd.read_sql(qry, db)
print('#records', len(df))
df.head()

### Group by

Repeat the 'select' for each group; works well with aggregate functions. Counting the observations by year, calculating the average ROA by industry-year, etc.

In [ ]:
qry ='''
select sich, fyear, count(*) as numObs
from funda 
where fyear >= 2018  and fyear <= 2022
and sich is not null
group by sich, fyear
;
'''
df = pd.read_sql(qry, db)
print('#records', len(df))
df.head()

In [ ]:
qry ='''
select sich, fyear, avg(ni/at) as roa_avg
from funda 
where fyear >= 2018  and fyear <= 2022
and sich is not null
group by sich, fyear
;
'''
df = pd.read_sql(qry, db)
print('#records', len(df))
df.head()

## `IN`, `NOT IN`

Use `IN` to select values from a list. You can actually bring the values from a dataframe.
Make sure to comma delimited them, and quote text. 

Use `NOT IN` to exclude observations with a value from the list.


In [ ]:
# set forces unique values, list turns it into a list again
yrs = list(set(list(df['fyear'])))
yrs

In [ ]:
yrs_commas = ",".join([str(item) for item in yrs])
yrs_commas

In [ ]:
qry ='''
select sich, fyear, avg(ni/at) as roa_avg
from funda 
where fyear in ({})
and sich is not null
group by sich, fyear
;
'''.format ( yrs_commas)

In [ ]:
print(qry)

In [ ]:
df = pd.read_sql(qry, db)
print('#records', len(df))
df.head()

## `distinct` keyword

Distinct is used to get unique records. For example, the list of gvkey-years in funda

In [ ]:
qry ='''
select distinct gvkey, fyear
from funda 
'''
df = pd.read_sql(qry, db)
print('#records', len(df))
df.head()

## `limit` keyword

The limit keyword is to limited the number of records.

In [ ]:
qry ='''
select distinct gvkey, fyear
from funda 
limit 3
'''
df = pd.read_sql(qry, db)
print('#records', len(df))
df.head()

## Joins

### Inner join

In [ ]:
qry = '''
select a.gvkey, a.datadate, a.fyear, a.sale, a.at, a.ni, b.lpermno as permno
from funda a, ccmxpf_linktable b
where a.fyear >= 1961 and a.fyear <=1999
and a.gvkey = b.gvkey
and b.linktype in ("LC", "LN", "LU", "LX", "LD", "LS")
and b.linkprim in ("C", "P")  
and a.datadate > b.linkdt and a.datadate <= linkenddt
and b.lpermno is not null
;
'''
df = pd.read_sql(qry, db)
print('#records', len(df))
df.head()

### Left join

With a left join, a `left join` is added after the first table and `where` becomes `on`

In [ ]:
qry = '''
select a.gvkey, a.datadate, a.fyear, a.sale, a.at, a.ni, b.lpermno as permno
from funda a left join ccmxpf_linktable b
on a.fyear >= 1961 and a.fyear <=1999
and a.gvkey = b.gvkey
and b.linktype in ("LC", "LN", "LU", "LX", "LD", "LS")
and b.linkprim in ("C", "P")  
and a.datadate > b.linkdt and a.datadate <= linkenddt
and b.lpermno is not null
;
'''
df = pd.read_sql(qry, db)
print('#records', len(df))
df.head()

## Self join

In a self join, the sample table is on the left and right. It is a great way to get lagged values.

In [ ]:
qry = '''
select a.gvkey, a.datadate, a.fyear, a.sale, b.sale as sale_prev, a.sale /b.sale -1 as sale_growth
from funda a, funda b
where a.fyear >= 1961 and a.fyear <=1999
and a.gvkey = b.gvkey
and a.fyear - 1 = b.fyear;
'''
df = pd.read_sql(qry, db)
print('#records', len(df))
df.head()

## Subqueries

Instead of a `from tablename`, you can write the tablename as a query.
Also the `IN` can be followed by a query (selecting one variable only though)

In the next example, Funda is joined with the linktable but using a different date (boy, 12 months before end of year).

Note how the subquery is labelled as `a`, and that `a.boy` is used in the filter to merge with `b`.

In [ ]:
qry = '''
select a.* , b.*
        from (
        select gvkey, datadate, fyear, epspi, (epspi < 0) as loss, date_add(datadate,  INTERVAL -12 MONTH) as boy,
        date_add(datadate, INTERVAL 4 MONTH) as eoy
            from funda 
            where 1961 <= fyear
            and fyear >= 1990
            limit 1000        
        ) a
        left join ccmxpf_linktable b
        on a.gvkey = b.gvkey
        and b.linktype in ("LC", "LN", "LU", "LX", "LD", "LS")
        and b.linkprim in ("C", "P")  
        and b.LINKDT <= a.boy 
        and a.boy <= b.LINKENDDT
        and b.lpermno IS NOT NULL
        ;
'''
df = pd.read_sql(qry, db)
print('#records', len(df))
df.head()